#Projeto de data sciencies em texto não supervisionado


# NIPS: Visualização de modelagem de tópicos

Alguns tópicos principais do NIPS de acordo com [wikipedia] (https://en.wikipedia.org/wiki/Conference_on_Neural_Information_Processing_Systems):

1. Aprendizado de máquina,
2. Estatísticas,
3. Inteligência artificial,
4. Neurociência computacional

No entanto, os tópicos estão dentro do mesmo domínio, o que torna mais difícil distingui-los. Aqui neste Kernel tentarei extrair alguns tópicos usando a alocação de Dirichlet latente __LDA__. Este tutorial apresenta um pipeline de processamento de linguagem natural de ponta a ponta, começando com dados brutos e passando pela preparação, modelagem e visualização do papel. Iremos abordar os seguintes pontos


1. Modelagem de tópico com ** LDA **
1. Visualização de modelos de tópicos com ** pyLDAvis **
1. Visualização dos resultados do LDA com ** t-SNE ** e ** bokeh **

In [1]:
#!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 6.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 196 kB/s 
     |████████████████████████████████| 11.3 MB 61.4 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=92e1719e3d051bf4790bfe35e65be7e1417f8a7f8e6217ae37ea554ea3195208
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [1]:
%pylab inline
import pandas as pd
import pickle as pk
from scipy import sparse as sp
pd.__version__

Populating the interactive namespace from numpy and matplotlib


'1.3.4'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive/'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
p_df = pd.read_csv('papers/Papers.csv')
docs = array(p_df['PaperText'])
docs[1]

'Learning with Symmetric Label Noise: The\nImportance of Being Unhinged\n\nBrendan van Rooyen∗,†\n∗\n\nAditya Krishna Menon†,∗\n\nThe Australian National University\n\n†\n\nRobert C. Williamson∗,†\n\nNational ICT Australia\n\n{ brendan.vanrooyen, aditya.menon, bob.williamson }@nicta.com.au\n\nAbstract\nConvex potential minimisation is the de facto approach to binary classification.\nHowever, Long and Servedio [2010] proved that under symmetric label noise\n(SLN), minimisation of any convex potential over a linear function class can result in classification performance equivalent to random guessing. This ostensibly\nshows that convex losses are not SLN-robust. In this paper, we propose a convex,\nclassification-calibrated loss and prove that it is SLN-robust. The loss avoids the\nLong and Servedio [2010] result by virtue of being negatively unbounded. The\nloss is a modification of the hinge loss, where one does not clamp at zero; hence,\nwe call it the unhinged loss. We show that the o

## Primeiro pré-processar deixando todo em minuscula e tokenizar o texto

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/LeonardoLins/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = str(docs[idx]).lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [6]:
docs = docs_preprocessor(docs)

### **Computar bigramas e trigramas :**
Sine tópicos são muito semelhantes, o que os faria distingui-los são frases ao invés de palavras únicas / individuais.

In [7]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

/opt/anaconda3/lib/python3.8/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [8]:
print(bigram)

Phrases<556123 vocab, min_count=10, threshold=10.0, max_vocab_size=40000000>


In [9]:
print(trigram)

Phrases<616916 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


### Remover Tokens comuns e pouco comuns**

In [10]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 39534
Number of unique words after removing rare and common words: 6001


Eliminando as palavras comuns e raras, acabamos com apenas cerca de 6% das palavras.

** Vetorizar dados: **
A primeira etapa é obter uma representação por trás das palavras de cada documento.
Passos
1-converter um corpus
doc2bow:converter documento (uma lista de palavras) no formato de saco de palavras = lista de (token_id, token_count) 2-tuplas. Cada palavra é considerada uma string tokenizada e normalizada (codificada em unicode ou em utf8). Nenhum outro pré-processamento é feito nas palavras do documento; aplique tokenização, lematização etc. antes de chamar esse método.


## precisso de 3 coisas para o algoritmo funcionar: Corpus um dicionario e o modelo 

In [11]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [12]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 6001
Number of documents: 403




Com o corpus de bag of words, podemos prosseguir para aprender nosso modelo de tópico a partir dos documentos.

# Entrenando LDA

In [13]:
from gensim.models import LdaModel

In [14]:
# Set training parameters.
num_topics = 4
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 1min 23s, sys: 600 ms, total: 1min 24s
Wall time: 1min 26s


# Como escolher a quantidade de tópicos?
__LDA__ é uma técnica não supervisionada, o que significa que não sabemos antes de executar o modelo quantos tópicos existem em nosso corpus. A coerência do tópico é uma das principais técnicas utilizadas para desestimar o número de tópicos. Você pode ler sobre isso [aqui.] (Http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf)

No entanto, usei a ferramenta de visualização LDA ** pyLDAvis **, tentei alguns tópicos e comparei os resultados. Quatro pareciam ser o número ideal de tópicos que separariam mais os tópicos.

In [15]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
pyLDAvis.__version__


'3.3.1'

In [16]:
gensimvis.prepare(model, corpus, dictionary)

/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.081555  0.020833       1        1  28.901733
1     -0.023986 -0.136990       2        1  25.234737
3      0.124469  0.052723       3        1  23.496025
2     -0.182038  0.063433       4        1  22.367505, topic_info=                  Term         Freq        Total Category  logprob  loglift
609             regret  1162.000000  1162.000000  Default  30.0000  30.0000
3230  gaussian_process   904.000000   904.000000  Default  29.0000  29.0000
3048  convergence_rate   927.000000   927.000000  Default  28.0000  28.0000
2180     convolutional   661.000000   661.000000  Default  27.0000  27.0000
3192            bandit   669.000000   669.000000  Default  26.0000  26.0000
...                ...          ...          ...      ...      ...      ...
1406          proposal   356.855427   579.067008   Topic4  -5.6213   1.0135
134       ground_truth   407.639159   797.063976   Topic4  -5.4883   0.8270
473          embedding   341.428976   619.398785   Topic4  -5.6655   0.9020
1062            filter   248.860122   376.963164   Topic4  -5.9818   1.0823
3353  generative_model   293.792600   795.344753   Topic4  -5.8158   0.5017

[294 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
4738      3  0.986850  accelerated_proximal
1228      2  0.972057            acceptance
3185      1  0.829671       active_learning
3185      2  0.055311       active_learning
3185      3  0.094179       active_learning
...     ...       ...                   ...
5637      4  0.986789     weakly_supervised
2132      1  0.982054        weighted_graph
2132      2  0.010912        weighted_graph
3015      1  0.012293               welfare
3015      3  0.983446               welfare

[794 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 3])

/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


** O que vemos aqui? **

** O painel esquerdo **, rotulado Mapa de distância intertópica, os círculos representam diferentes tópicos e a distância entre eles. Tópicos semelhantes aparecem mais próximos e tópicos diferentes mais distantes.
O tamanho relativo do círculo de um tópico no gráfico corresponde à frequência relativa do tópico no corpus.
Um tópico individual pode ser selecionado para um exame mais detalhado clicando em seu círculo ou inserindo seu número na caixa "tópico selecionado" no canto superior esquerdo.
 
** O painel direito ** inclui o gráfico de barras dos 30 principais termos. Quando nenhum tópico é selecionado no gráfico à esquerda, o gráfico de barras mostra os 30 termos mais "salientes" no corpus. A saliência de um termo é uma medida de quão frequente o termo é no corpus e quão "distinto" ele é na distinção entre diferentes tópicos.
Selecionar cada tópico à direita modifica o gráfico de barras para mostrar os termos "relevantes" para o tópico selecionado.
A relevância é definida como no rodapé 2 e pode ser ajustada pelo parâmetro $ \ lambda $, menor $ \ lambda $ dá maior peso à distinção do termo, enquanto $ \ lambda $ s maior corresponde à probabilidade da ocorrência do termo por tópicos.

Portanto, para ter uma noção melhor dos termos por tópico, usaremos $ \ lambda $ = 0.

** Como avaliar nosso modelo? **
Então, novamente, uma vez que não há base para aqui, temos que ser criativos na definição de maneiras de avaliar. Eu faço isso em duas etapas:

1. divida cada documento em duas partes e veja se os tópicos atribuídos a eles são semelhantes. => quanto mais semelhante, melhor
2. comparar documentos escolhidos aleatoriamente entre si. => quanto menos semelhante melhor

transformando os dados


De cima, é possível inspecionar cada tópico e atribuir um rótulo interpretável por humanos a ele. Aqui, eu os rotulei da seguinte maneira:

In [17]:
from collections import OrderedDict
def get_doc_topic_dist(model, corpus, kwords=False):
    
    '''
   A transformação LDA, para cada doc retorna apenas tópicos com peso diferente de zero
     Esta função faz uma transformação de matriz de documentos no espaço do tópico.
    '''
    top_dist =[]
    keys = []

    for d in corpus:
        tmp = {i:0 for i in range(num_topics)}
        tmp.update(dict(model[d]))
        vals = list(OrderedDict(tmp).values())
        top_dist += [array(vals)]
        if kwords:
            keys += [array(vals).argmax()]

    return array(top_dist), keys

In [18]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, atopic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [19]:
topic_summaries = []
print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
for i in range(num_topics):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )
#     print tmp[:5]
    topic_summaries += [tmp[:5]]
    print

term                 frequency

Topic 0 |---------------------

data_set             0.005
convergence_rate     0.004
line_search          0.004
active_learning      0.004
strongly_convex      0.004
random_walk          0.004
query                0.004
empirical_risk       0.003
primal_dual          0.003
first_order          0.003
Topic 1 |---------------------

gaussian_process     0.008
variational_inference 0.006
tensor               0.006
matrix_completion    0.004
gibbs_sampling       0.004
posterior_distribution 0.004
gibbs                0.004
topic_model          0.004
sampler              0.004
markov_chain         0.003
Topic 2 |---------------------

sample_complexity    0.005
submodular           0.005
vertex               0.005
step_size            0.004
convergence_rate     0.004
probability_least    0.004
running_time         0.004
graphical_model      0.004
sample_size          0.004
markov_chain         0.004
Topic 3 |---------------------

regret               0.008
